In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen

import time
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [3]:
print(len(masks.ImageId))
print(len(masks.EncodedPixels))

231723
231723


In [4]:
num_examples = len(masks.ImageId)
t_per_file = []
start = time.time()
list_error_index = []
for index, imgId in enumerate(masks.ImageId) :
    if index%100==0 or index==num_examples-1:
        clear_output(wait=True)
        t_per_file.append((time.time() - start) / 100)
        eta = np.mean(t_per_file) * (num_examples - index)
        print("ETA:", round(eta/60,2), end=" minutes ==>  ")
        print(index, "/", num_examples, end" : ")
        print(len(list_error_index), "errors detected")
        start = time.time()
    img_path = os.path.join(train_image_dir, imgId)
    try:
        img = imread(img_path)
    except:
        list_error_index.append(index)

ETA: 0.0 minutes ==>  231722 / 231723


In [5]:
np.savetxt("log_list_error.txt", list_error_index)

In [6]:
new_dataframe = masks.drop(list_error_index, axis=0)
print("new length =", len(new_dataframe))
print("diff length :", len(masks)-len(new_dataframe))
print("errors length :", len(list_error_index))

new length = 231722
diff length : 1
errors length : 1


In [7]:
new_dataframe.to_csv(ship_dir + "train_ship_segmentations_v2.csv")